In [84]:
import numpy as np
import pandas as pd

In [85]:
df = pd.read_html("YorkCityStats20.html")
sn = pd.read_html("YorkCityLiveSquadNumbers.html")
sns = sn[0]
# match_squad_shirt_numbers = sn[0]["No."].tolist()

In [86]:
dfs = df[0]
dfs
sns


,Position Selected,Name,No.
0,GK,Marco Bleve,1
1,DCR,Taylor Moore,28
2,DC,Sam Stubbs,13
3,DCL,Kean Bryan,35
4,WBR,Diogo Dalot,36
5,WBL,Lewie Coyle,27
6,MCR,Kyllan RamÃ©,8
7,MC,Ilias Koutsoupias,5
8,MCL,Boubakary SoumarÃ©,34
9,STCR,Felix Nmecha,33


In [87]:
# Takes in players of type dataframe, only one including position selected and shirt no
def findPlayingSquad(players):
    shirts = []
    for i in range(0,len(players)):
        sub_names = ["S1","S2","S3","S4","S5","S6","S7","S8","S9","S10","S11","S12"]
        curr_pos = players["Position Selected"][i]
        curr_shirt = int(players["No."][i])
        
        if (not (curr_pos in sub_names)) and (type(curr_pos) != float):
            shirts.append(curr_shirt)
            
    return shirts

In [88]:
###    ATT corners & Throw-Ins

# FP Attack far post, NP Attack at Near Post, GF Go Forward, MK Mark keeper, LNR Lurk at Near Post
# LFP Lurk at Far Post (InTheBox - itb describes all 6 roles)
itb = ["Jum","Str","Hea","OtB","Bra","Fin","Dec"]

# EDGE Attack Ball from the Edge (OutsideTheBox - otb)
otb = ["Jum","Str","Hea","OtB","Ant","Fin"]

# LOA Lurk Outside Area - loa
loa = ["Fin","Lon","Fir","Pas","Tec","Dec"]

# SB Stay Back, SBI Stay Back If Needed
sb = ["Acc","Tea","Mar","Tck","Pos","Dec"]

# CS Come Short
cs = ["Cro","Dri","Pas","Tec","Ant","Dec"]


###    ATT free kicks

# All other same as ATT Corners

# SWT Stand With Taker
swt = ["Cro","Dri","Pas","Tec","Ant","Dec"]



### DEF Corners & Throw-Ins

# 6F 6FC 6C 6NC Zonally Mark Six Yard .. , GB Go Back, FP Mark Far Post, basic defense - bd
bd = ["Jum","Str","Hea","Pos","Bra","Cnt"]

# NP Mark Near Post - mnp and not np because it collide with numpy np
mnp = ["Jum","Str","Hea","Ant","Bra","Cnt"]

# MM Man Mark, MT Mark Tall Player
mm = ["Jum","Str","Hea","Mar","Bra","Cnt"]

# EA Edge of Areaa
edge = ["Ant","Acc","Cnt","Str","Agg","Pas"]

# CD Close Down Corner
cd = ["Ant","Mar","Cnt","Str","Agg","Tck"]

# SF Stay Forward
sf = ["Acc","Agi","Pac","Str","Bal","Ant"]

### DEF Free Kicks

# All other same as DEF Corners & Throw-Ins

# WAL Form Wall
fw = ["Jum","Bra","Cnt","Tea"]


In [89]:
def find_score(role):
    scores = {}
    for i in dfs.index:
        score = 0
        for attr in role:
            score += dfs[attr][i]
            name = dfs["Name"][i]  
        if role in [itb, otb, bd, mnp, mm]:
            score += int(dfs["Height"][i][0:3])
        scores.update({name:score})
    return scores


def get_score(set_piece):
    i = 0
    for role in set_piece:
        set_piece[i] = find_score(role)
        i += 1
    return set_piece

In [90]:
att_corner = [itb, otb, loa, sb, cs] 
att_corner_str = ["itb","otb","loa","sb", "cs"]


att_FK = att_corner + [swt]
att_FK_str = att_corner_str + ["swt"]


def_corner_ti = [bd,mnp,mm,edge,cd,sf]
def_corner_ti_str = ["bd","mnp","mm","edge","cd","sf"]


def_FK = def_corner_ti + [fw]
def_FK.pop(1)
def_FK.pop(4)
def_FK_str = def_corner_ti_str + ["fw"]
def_FK_str.pop(1)
def_FK_str.pop(4)



att_corner = get_score(att_corner)
att_FK = get_score(att_FK)

def_corner_ti = get_score(def_corner_ti)
def_FK = get_score(def_FK)


In [91]:
def parse_input(user_input):
    # Add an ASSERT statement to make sure input can't break the program
    pts = []
    squad_no = ''
    for i in user_input:
        if i == ',':
            squad_no = int(squad_no)
            pts.append(squad_no) 
            squad_no = ''
            continue
        else:
            squad_no += i
        
    return pts
    

In [92]:
# user = input("Give me the squad numbers of the players you want to sort:") 
# # 4,7,17,12,9,28,21,24
# players_to_score = parse_input(user)

all_players = dfs["No."].tolist()
# players_to_score = [24,18,25,2,3,7,12,28,21,6]
# players_to_score = all_players
players_to_score = findPlayingSquad(sns)


In [93]:
def sorting(players,set_piece, set_piece_str):
    new_scores = []
    sps_index = 0
    sdf = pd.DataFrame()
    for role in set_piece:
        new_names = []
        new_scores = []
        for shirt_no in players:
            name = dfs.loc[dfs['No.'] == shirt_no]["Name"].values[0] 
            new_names.append(name)
            new_scores.append(role[name]) 
        sdf["Name"] = new_names 
        sdf[set_piece_str[sps_index]] =  new_scores
        sps_index += 1
    return sdf
    
        
          

In [94]:
att_corner_sort = sorting(players_to_score,att_corner, att_corner_str)
att_FK_sort = sorting(players_to_score, att_FK, att_FK_str)
# att_TI_sort = sorting(players_to_score,att_TI,att_TI_str)

def_corner_ti_sort = sorting(players_to_score, def_corner_ti, def_corner_ti_str)
def_FK_sort = sorting(players_to_score, def_FK, def_FK_str)


In [95]:
att_corner_sort.sort_values('itb',ascending = False)

,Name,itb,otb,loa,sb,cs
3,Kean Bryan,276,258,70,85,76
8,Boubakary SoumarÃ©,276,262,77,79,69
1,Taylor Moore,272,255,60,81,64
10,Alexandros Chrysikos,271,255,72,59,71
7,Ilias Koutsoupias,270,251,75,91,74
2,Sam Stubbs,266,252,52,71,57
9,Felix Nmecha,262,251,83,65,74
6,Kyllan RamÃ©,261,249,79,70,77
4,Diogo Dalot,259,246,65,73,79
5,Lewie Coyle,249,234,57,75,70


In [96]:
att_corner_sort.sort_values('otb',ascending = False)

,Name,itb,otb,loa,sb,cs
8,Boubakary SoumarÃ©,276,262,77,79,69
3,Kean Bryan,276,258,70,85,76
1,Taylor Moore,272,255,60,81,64
10,Alexandros Chrysikos,271,255,72,59,71
2,Sam Stubbs,266,252,52,71,57
7,Ilias Koutsoupias,270,251,75,91,74
9,Felix Nmecha,262,251,83,65,74
6,Kyllan RamÃ©,261,249,79,70,77
4,Diogo Dalot,259,246,65,73,79
5,Lewie Coyle,249,234,57,75,70


In [97]:
att_corner_sort.sort_values('loa',ascending = False)

,Name,itb,otb,loa,sb,cs
9,Felix Nmecha,262,251,83,65,74
6,Kyllan RamÃ©,261,249,79,70,77
8,Boubakary SoumarÃ©,276,262,77,79,69
7,Ilias Koutsoupias,270,251,75,91,74
10,Alexandros Chrysikos,271,255,72,59,71
3,Kean Bryan,276,258,70,85,76
4,Diogo Dalot,259,246,65,73,79
1,Taylor Moore,272,255,60,81,64
5,Lewie Coyle,249,234,57,75,70
2,Sam Stubbs,266,252,52,71,57


In [98]:
att_corner_sort.sort_values('cs',ascending = False)

,Name,itb,otb,loa,sb,cs
4,Diogo Dalot,259,246,65,73,79
6,Kyllan RamÃ©,261,249,79,70,77
3,Kean Bryan,276,258,70,85,76
7,Ilias Koutsoupias,270,251,75,91,74
9,Felix Nmecha,262,251,83,65,74
10,Alexandros Chrysikos,271,255,72,59,71
5,Lewie Coyle,249,234,57,75,70
8,Boubakary SoumarÃ©,276,262,77,79,69
1,Taylor Moore,272,255,60,81,64
2,Sam Stubbs,266,252,52,71,57


In [16]:
# att_FK_sort.sort_values('swt',ascending = False)
# def_corner_ti_sort.sort_values('bd',ascending = False)
# def_FK_sort.sort_values('mm',ascending = False)

In [46]:
def bestFKtakers(players):
    df_left = pd.DataFrame()
    df_right = pd.DataFrame()
    names_left = []
    names_right = []
    scores_left = []
    scores_right = []
    feet_left = []
    feet_right = []
    for shirt_no in players:
        score = 0
        name = dfs.loc[dfs['No.'] == shirt_no]
        score = name['Fre'].values[0]+ 0.8*name['Cro'].values[0]+ \
        0.8*name['Lon'].values[0]+0.4*name['Pas'].values[0]+ \
        0.5*name['Tec'].values[0]+0.65*name['Cmp'].values[0]+ \
        0.55*name['Cnt'].values[0]
        
        foot = dfs.loc[dfs["Name"] == name.values[0][0]]["Preferred Foot"].values[0]
        if foot in ["Left","Left Only","Either"]:
            names_right.append(name["Name"].values[0])
           
            scores_right.append(score)
            feet_left.append(foot)
        else:
            names_left.append(name["Name"].values[0])
            scores_left.append(score)
            feet_right.append(foot)

    
    df_left["Name"] = names_left
    df_left["Preferred Foot"] = feet_right
    df_left["Left FK Taker Score"] = scores_left
    
    
    df_right["Name"] = names_right
    df_right["Preferred Foot"] = feet_left
    df_right["Right FK Taker Score"] = scores_right
    return (df_left.sort_values('Left FK Taker Score',ascending = False),df_right.sort_values('Right FK Taker Score',ascending = False))
        

In [47]:
left_FK_takers,right_FK_takers = bestFKtakers(all_players)
print(left_FK_takers)

                          Name Preferred Foot  Left FK Taker Score
14                 Tommy Doyle     Right Only                63.25
5            Ilias Koutsoupias     Right Only                56.35
15             Hakeeb Adelakun          Right                52.50
13                Felix Nmecha          Right                52.25
9           Boubakary SoumarÃ©          Right                50.60
3               Conor Coventry          Right                50.00
10              Ceri Robertson          Right                49.50
1              MartÃ­n Montoya          Right                49.00
6                   Will Marsh          Right                48.40
11                 Diogo Dalot          Right                47.50
7                Jordan Hugill          Right                46.00
2                 Taylor Moore     Right Only                43.20
4   Alexandros Sakellaropoulos     Right Only                42.10
0                  Marco Bleve          Right                4

In [48]:
print(right_FK_takers)

                   Name Preferred Foot  Right FK Taker Score
5                 Diego           Left                 67.30
2          Kyllan RamÃ©      Left Only                 57.55
1            George Cox           Left                 55.30
7           Noam Emeran           Left                 53.95
4  Alexandros Chrysikos         Either                 48.50
0            Kean Bryan           Left                 48.10
3          Will Jackson      Left Only                 45.80
6           Lewie Coyle         Either                 41.70


In [19]:
def bestCornerTakers(players):
    df = pd.DataFrame() 
    df2 = pd.DataFrame()
    names_left = []
    names_right = []
    feet_left = []
    scores_left = []
    scores_right = []
    feet_right = []
    for shirt_no in players:
        score = 0
        name = dfs.loc[dfs['No.'] == shirt_no]
        score = name['Cor'].values[0]+ 0.8*name['Cro'].values[0]+ \
        0.8*name['Pas'].values[0]+0.7*name['Tec'].values[0]
        
      
        foot = dfs.loc[dfs["Name"] == name.values[0][0]]["Preferred Foot"].values[0]
        if foot in ["Left","Left Only","Either"]:
            names_left.append(name["Name"].values[0])
            scores_left.append(score)
            feet_left.append(foot)
        else:
            names_right.append(name["Name"].values[0])
            scores_right.append(score)
            feet_right.append(foot)
 
    df["Name"] = names_left
    df["Preferred Foot"] = feet_left
    df["Right Corner Taker Score"] = scores_left
    
    df2["Name"] = names_right
    df2["Preffered Foot"] = feet_right
    df2["Left Corner Taker Score"] = scores_right
    
    return (df.sort_values('Right Corner Taker Score', ascending = False), df2.sort_values('Left Corner Taker Score', ascending = False))
    

In [20]:
best_corner_taker_right,best_corner_taker_left = bestCornerTakers(all_players)

# best_corner_taker_left,best_corner_taker_right = bestCornerTakers(players_to_score)


In [21]:
print(best_corner_taker_right)

                   Name Preferred Foot  Right Corner Taker Score
5                 Diego           Left                      48.3
1            George Cox           Left                      41.6
2          Kyllan RamÃ©      Left Only                      40.2
3          Will Jackson      Left Only                      39.6
7           Noam Emeran           Left                      38.2
0            Kean Bryan           Left                      34.5
4  Alexandros Chrysikos         Either                      32.8
6           Lewie Coyle         Either                      32.3


In [24]:
print(best_corner_taker_left)

                          Name Preffered Foot  Left Corner Taker Score
14                 Tommy Doyle     Right Only                     46.9
11                 Diogo Dalot          Right                     38.7
15             Hakeeb Adelakun          Right                     38.1
5            Ilias Koutsoupias     Right Only                     36.5
1              MartÃ­n Montoya          Right                     36.2
3               Conor Coventry          Right                     35.7
13                Felix Nmecha          Right                     35.1
4   Alexandros Sakellaropoulos     Right Only                     32.7
9           Boubakary SoumarÃ©          Right                     31.3
10              Ceri Robertson          Right                     31.3
6                   Will Marsh          Right                     31.0
0                  Marco Bleve          Right                     29.5
7                Jordan Hugill          Right                     29.0
2     

In [122]:
def cornerAlgo(players):
    corner_taker,cs, otb, itb = [() for i in range(5)]

    
    best_corner_taker_left.loc[best_corner_taker_left["Left Corner Taker Score"].idxmax()]
    
    
    
    
    
    

In [123]:
print(cornerAlgo(players_to_score))

ValueError: too many values to unpack (expected 4)

In [124]:
example = [("Name",23),("SeconD",25),("Thir",54)]
print("Name" in example[0])

True


In [493]:
print(np.random.randint(low = 2,high = 4))
# np.random.randint(2, size=10)

1
